<a href="https://colab.research.google.com/github/alexchen1999/covid-19-sample-strain-classification/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import numpy as np
import os
import torch.nn.functional as F
import torch.optim as optim
import sys
import pandas as pd
import re
from torch.utils.data import (TensorDataset, DataLoader, Dataset)
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
# from Bio import SeqIO 

from sklearn.model_selection import train_test_split

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Specify hyperparameters ?

In [ ]:
# Read pandas csv
df = pd.read_csv('spike_seq_one_hots_100.csv')
df.head()

,Accession,Seq,Label
0,OK114603.1,"[[0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...",Delta
1,OL738471.1,"[[0.0, 0.0, 0.0, 1.0], [0.0, 1.0, 0.0, 0.0], [...",Delta
2,OM790020.1,"[[0.0, 0.0, 0.0, 1.0], [0.0, 1.0, 0.0, 0.0], [...",Omicron
3,OM769650.1,"[[1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0....",Omicron
4,OM639250.1,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...",Omicron


In [ ]:
def str2array(s):
    return torch.from_numpy(np.array(json.loads(s)))

df1=pd.read_csv('spike_seq_one_hots_100.csv', converters={'Seq': str2array})
print(df1.iloc[0]['Seq'])
print(type(df1.iloc[0]['Seq']))
print(df1.iloc[0]['Seq'].shape)
print(df1.shape)

tensor([[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.]], dtype=torch.float64)
<class 'torch.Tensor'>
torch.Size([3822, 4])
(400, 3)


In [ ]:
class MyDataset(Dataset):
 
  def __init__(self, X, y):
    self.X = X
    self.y = y
 
  def __len__(self):
    return len(self.X)
   
  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

In [ ]:
# Train Test Split
X = df1.iloc[:, 1:2]
y = df1.iloc[:, 2:3]
print(X.shape)
print(y.shape)
print(type(X))
print(type(y))
y.head()
train_inputs, test_inputs, train_labels, test_labels = train_test_split(
   X, y, test_size=0.1, random_state=42)

# Load data to PyTorch DataLoader
train_dataset = MyDataset(train_inputs, train_labels)
test_dataset = MyDataset(test_inputs, test_labels)
train_dataloader = DataLoader(train_dataset, batch_size=40)
test_dataloader = DataLoader(test_dataset, batch_size=4)

(400, 1)
(400, 1)
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [ ]:
# Training
def train(net, dataloader, epochs=1, lr=0.01, momentum=0.9, decay=0.0, verbose=1):
  ''' Trains a neural network. Returns a 2d numpy array, where every list 
  represents the losses per epoch.
  '''
  net.to(device)
  losses_per_epoch = []
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum, weight_decay=decay)
  for epoch in range(epochs):
    sum_loss = 0.0
    losses = []
    for i, batch in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = batch[0].to(device), batch[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize 
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        losses.append(loss.item())
        sum_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            if verbose:
              print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, sum_loss / 100))
            sum_loss = 0.0
    # print(len(losses))
    losses_per_epoch.append(np.mean(losses))
  return losses_per_epoch

In [ ]:
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4908339/
# In-channels should be L * 4 where L is sequence length, 3822
# RGB is 3 * w * h
# DNA is 
ex = [['in_channels_conv1', 'out_channels_conv1', 'in_channels_conv2', 'out_channels_conv2', 'in_channels_fc1', 'kernel_size', 'stride', ('train_acc'), ('test_acc')],
      [60, 40, 20, 40, 40, 3, 3], #1
      [60, 240, 120, 480, 480, 3, 3], #2 
      [60, 480, 240, 960, 960, 3, 3, 0, (0.973528), (0.918495)], #3
      [60, 480, 240, 960, 960, 6, 3, 0, (0.998607), (0.962382)], #4
      [(3822, 4), (3822, 4), 240, 960, 960, 6, 24,  1], #5 BEST ONE! :)
      [10, 80, 40, 160, 160, 6, 3,  0.01, (0.9738766980146291), (0.9561128526645768)], #6
      [60, 480, 240, 960, 960, 6, 3,  0.02, (0.9641239986067572), (0.934169278996865)], #7
]
i = 5

FEATURE_SIZE = ex[i][0]
print(FEATURE_SIZE)

print(ex[0])
print(ex[i])
print(ex[i][0])

class DSC_by_CNN(nn.Module):
    def __init__(self, feature_size):


        super(DSC_by_CNN, self).__init__()
        self.feature_size = feature_size

        self.conv1 = nn.Conv1d(in_channels=ex[i][0],
                               out_channels=ex[i][1], 
                               kernel_size=ex[i][5],
                               stride = ex[i][6],
                               padding = 1)
        self.conv2 = nn.Conv1d(in_channels=ex[i][2],
                               out_channels=ex[i][3], 
                               kernel_size=ex[i][5],
                               stride = ex[i][6],
                               padding = 1)


        # Activation
        self.act = nn.ReLU()

        # Pooling
        self.pool = nn.MaxPool2d(3, 2, 1)

        # fully connected layer
        self.fc1 = nn.Linear(ex[i][4], 100)

        # dropout
        self.drop = nn.Dropout(p=0.5)

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features        


    def forward(self, x):
        x = torch.tensor(x).to(torch.int64)

        # Encode. 
        # Output shape: (b, max_len, embedded_dim)
        x = self.encoder(x)

        # Permute shuffles dimensions. Do this to satisfy requirments of nn.Conv1d
        # Output shape: (b, embedded_dim, max_len)
        x = x.permute(0, 2, 1)

        # convolutional layer 1
        x = self.conv1(x)
        x = self.act(x)
        x = self.pool(x)

        # convolutional layer 2
        x = self.conv2(x)
        x = self.act(x)
        x = self.pool(x)

        # flattening
        x = x.view(-1, self.num_flat_features(x))

        # fully linear layer + dropout
        x = self.drop(x)
        # x = self.softm(x)
        x = self.fc1(x)

        return x

(3822, 4)
['in_channels_conv1', 'out_channels_conv1', 'in_channels_conv2', 'out_channels_conv2', 'in_channels_fc1', 'kernel_size', 'stride', 'train_acc', 'test_acc']
[(3822, 4), (3822, 4), 240, 960, 960, 6, 24, 1]
(3822, 4)


In [ ]:
# Creating net and training

net = DSC_by_CNN(FEATURE_SIZE)
losses = train(net, train_dataloader, epochs=50, lr=0.01, decay = ex[i][-3])

# Evaluation
plot_losses(losses, title = "Splice Dataset: Convolutional Network Loss")
print_eval(net, train_dataloader, test_dataloader)
print()

TypeError: ignored